In [1]:
import pandas as pd
import os

class ReadDataframe:
    
    def __init__(self, df, name):
        self.df = df
        self.name = os.path.splitext(os.path.basename(name))[0]
        self.dir = os.path.dirname(name)
        self.pointsP1 = []
        self.pointsP2 = []
        self.heightP1 = []
        self.heightP2 = []
        self.hX = []
        self.hY = []
        self.trayPoints = []
        self.P1 = []
        self.P2 = []
        self.baseplate = 'None'
        self.sensor = 'None'
        self.pcb = 'None'

    def getColumn(self, name = ''):
        for col in self.df.columns:
            if (self.df[col] == name).any():
                return col
        
        return ''
        
    def getComponents(self):
        if 'cuw' in self.name.casefold(): self.baseplate = 'CuW'
        elif 'pcb' in self.name.casefold(): self.baseplate = 'PCB'
            
        if '200' in self.name.casefold(): self.sensor = '200'
        elif '300' in self.name.casefold(): self.sensor = '300'
            
        if 'ld' in self.name.casefold(): self.pcb = 'LD'
        elif 'hd' in self.name.casefold(): self.pcb = 'HD'
        
    # Function to sort hte list by second item of tuple
    def sortTuple(self, tup):
        if len(tup) == 0: return []
        # reverse = True (Sorts in descending order for Y coordinate)
        # reverse = None (Sorts in ascending order for X coordinate)
        # key is set to sort using first or second element of 
        # sublist lambda has been used 
        tup.sort(key = lambda x: x[1], reverse=True)
        tup1, tup2 = tup[:2], tup[2:]
        tup1.sort(key = lambda x: x[0])
        tup2.sort(key = lambda x: x[0])
    
        new = [tup1[0], tup2[0], tup1[1], tup2[1]]
    
        return new 
    
    def readComponentHeights(self):
        self.getComponents()
        attribute = self.getColumn('Attribute')
        feature = self.getColumn('Feature')
        actual = self.getColumn('Actual')
        X = self.df.index[self.df[attribute] == 'Z'].tolist()
        for x in X:
            if 'thickness' in self.df[feature][x].casefold():
                if 'pm1' in self.df[feature][x].casefold():
                    self.heightP1.append(float(self.df[actual][x]))
                elif 'pm2' in self.df[feature][x].casefold():
                    self.heightP2.append(float(self.df[actual][x]))
                else:
                    self.heightP1.append(float(self.df[actual][x]))
        
        self.hX = [83.565, 45.912, -0.651, -9.592, -28.199, -48.788, -44.552, -28.132, -12.698, 3.772, 43.071, 80.717, 
             96.944, 113.55, 127.835, 131.54, 113.796, 92.786, 42.359, -2.788, 4.469, 42.661, 78.847, 83.524, 42.317]
        self.hY = [-7.422, -7.876, -8.36, -1.118, 33.167, 67.776, 86.943, 115.068, 141.539, 154.474, 154.6, 155.388, 
             141.565, 113.219, 87.506, 65.722, 36.932, -1.446, 25.756, 49.506, 95.797, 120.628, 105.42, 48.884, 70.63]
    
    def readComponentFiducials(self):
        attribute = self.getColumn('Attribute')
        feature = self.getColumn('Feature')
        actual = self.getColumn('Actual')
        X = self.df.index[self.df[attribute] == 'X'].tolist()
        for x in X:
            if 'fiducial' in self.df[feature][x].casefold():
                if 'p1' in self.df[feature][x].casefold():
                    self.pointsP1.append((float(self.df[actual][x]), float(self.df[actual][x + 1])))
                elif 'p2' in self.df[feature][x].casefold():
                    self.pointsP2.append((float(self.df[actual][x]), float(self.df[actual][x + 1])))
    
        self.pointsP1 = self.sortTuple(self.pointsP1)
        self.pointsP2 = self.sortTuple(self.pointsP2)

    
    def readTrayFiducials(self):
        #Get index of all points that correspond to the two reference frames
        label = self.getColumn('Label')
        actual = self.getColumn('Actual')
        trayIdx = self.df.index[self.df[label] == 'TrayFiducial2Fine2.X'].tolist()[0]
        P1IdxCenter = self.df.index[self.df[label] == 'P1CenterPin.X'].tolist()[0]
        P1IdxOffCenter = self.df.index[self.df[label] == 'P1OffcenterPin.X'].tolist()[0]
        P2IdxCenter = self.df.index[self.df[label] == 'P2CenterPin.X'].tolist()[0]
        P2IdxOffCenter = self.df.index[self.df[label] == 'P2OffcenterPin.X'].tolist()[0]

        #Organize such that the Center is always the first element
        #Off center is always the second element
        self.trayPoints = [(0., 0.), (float(self.df[actual][trayIdx]), float(self.df[actual][trayIdx + 1]))]
        self.P1 = [(float(self.df[actual][P1IdxCenter]),    float(self.df[actual][P1IdxCenter + 1])), 
              (float(self.df[actual][P1IdxOffCenter]), float(self.df[actual][P1IdxOffCenter + 1]))]
        self.P2 = [(float(self.df[actual][P2IdxCenter]),    float(self.df[actual][P2IdxCenter + 1])), 
              (float(self.df[actual][P2IdxOffCenter]), float(self.df[actual][P2IdxOffCenter + 1]))]